In [ ]:
import queue
import threading
import pyrealsense2
from realsense_camera import *
import time
import cv2
from ObjectDetector import ObjectDetector
from kuka import Kuka
from openshowvar import *
import numpy as np
from ultralytics import YOLO
import supervision as sv
import numpy as np

print('go')
robot = openshowvar(ip = '192.168.17.2', port = 7000)
kuka = Kuka(robot)

kuka.read_cartessian()

kuka.set_base(17)
kuka.set_tool(11)

# Set speed (%)
kuka.set_speed(100)
print('ready')

model = YOLO("Battery_1.pt")


# Function to clear the queue
def clearQueue(dataQueue):
    while not dataQueue.empty():
        dataQueue.get()

# Function for the child thread to process coordinates
def childProcess(stopEvent, dataQueue):
    while not stopEvent.is_set():
        try:
            coord = dataQueue.get(timeout=1)     
        except queue.Empty:
            continue
        if coord is not None:
            trajectory_arr = []
            trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
            trajectory_arr.append(np.array([50, 900, 300, 0, 0, 180]))
            trajectory_arr = np.array(trajectory_arr)
            # Отправка траектории роботу
            kuka.lin_continuous(trajectory_arr) 
            # Построение траектории
            trajectory_arr = []
            trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
            trajectory_arr.append(np.array([coord['x_sort'][0][2], 900, 450, 0, 0, 180]))
            trajectory_arr = np.array(trajectory_arr) 
            # Отправка траектории роботу
            kuka.lin_continuous(trajectory_arr) 
            if (4-(int(time.time())-coord["second"]))<0:
                clearQueue(dataQueue)
            else:
                time.sleep(4-(int(time.time())-coord["second"]))
            kuka.close_grip() 
            if coord['currentClass']==0:
                trajectory_arr = []
                trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
                trajectory_arr.append(np.array([coord['x_sort'][0][2], 900, 200, 0, 0, 180]))
                trajectory_arr.append(np.array([-200, 500, 200, 90, 0, 180]))
                trajectory_arr = np.array(trajectory_arr)
                # Отправка траектории роботу
                kuka.lin_continuous(trajectory_arr)  
                kuka.open_grip
                # Построение траектории
                trajectory_arr = []
                trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
                trajectory_arr.append(np.array([coord['x_sort'][0][2], 900, 300, 0, 0, 180]))
                trajectory_arr = np.array(trajectory_arr)
                # Отправка траектории роботу
                kuka.lin_continuous(trajectory_arr) 
                clearQueue(dataQueue)
            if coord['currentClass']==1:
                trajectory_arr = []
                trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
                trajectory_arr.append(np.array([coord['x_sort'][0][2], 900, 200, 0, 0, 180]))
                trajectory_arr.append(np.array([-200, 300, 200, 90, 0, 180]))
                trajectory_arr = np.array(trajectory_arr)
                # Отправка траектории роботу
                kuka.lin_continuous(trajectory_arr)  
                kuka.kuka.open_grip
                # Построение траектории
                trajectory_arr = []
                trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
                trajectory_arr.append(np.array([coord['x_sort'][0][2], 900, 300, 0, 0, 180]))
                trajectory_arr = np.array(trajectory_arr)
                # Отправка траектории роботу
                kuka.lin_continuous(trajectory_arr) 
                clearQueue(dataQueue)
            else:
                trajectory_arr = []
                trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
                trajectory_arr.append(np.array([coord['x_sort'][0][2], 900, 200, 0, 0, 180]))
                trajectory_arr.append(np.array([0, 100, 200, 90, 0, 180]))
                trajectory_arr.append(np.array([200, 100, 200, 90, 0, 180]))
                trajectory_arr = np.array(trajectory_arr)
                # Отправка траектории роботу
                kuka.lin_continuous(trajectory_arr)  
                kuka.vacuum_off()
                # Построение траектории
                trajectory_arr = []
                trajectory_arr.append(np.array([kuka.x_cartessian, kuka.y_cartessian, kuka.z_cartessian, kuka.A_cartessian, kuka.B_cartessian, kuka.C_cartessian]))
                trajectory_arr.append(np.array([0, 100, 200, 90, 0, 180]))
                trajectory_arr.append(np.array([coord['x_sort'][0][2], 900, 300, 0, 0, 180]))
                trajectory_arr = np.array(trajectory_arr)
                # Отправка траектории роботу
                kuka.lin_continuous(trajectory_arr) 
                clearQueue(dataQueue)
        else:
            clearQueue(dataQueue)

if __name__ == "__main__":
    # Open the video capture device
    cap = cv2.VideoCapture(1)  
    # Create an instance of ObjectDetector
    detector = ObjectDetector()
    # Create an Event object to signal the child thread to stop
    stopEvent = threading.Event()
    # Create a queue to communicate coordinates between threads
    dataQueue = queue.Queue()
    coord = None
    dataQueue.put(coord)
    # Create the child thread
    childThread = threading.Thread(target=childProcess, args=(stopEvent, dataQueue,))
    # Start the child thread
    childThread.start()
    coordinat=[]
    while True:
        ret, frame = cap.read()
        frame = frame[120:370,:]
        coord = detector.detect(frame,model)    
        print(coord)
        if coord['isEmpty']==True:
            coordinat.append(coord)    
            while dataQueue.empty()==True and coordinat!=[]:
                if coordinat[0]['second']!=None: 
                    if int(time.time())-coordinat[0]['second']<=5:
                        dataQueue.put(coordinat[0])
                        break
                    else:
                        coordinat.pop(0)
                else:
                    coordinat.pop(0) 
                continue

        # Display the frame
        cv2.imshow("Frame", frame)
               
        # Check for the 'q' key press to stop the program
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            stopEvent.set()
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()

go
ready



0: 640x640 (no detections), 241.0ms
Speed: 10.2ms preprocess, 241.0ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)



{'x_sort': None, 'currentClass': None, 'isEmpty': False, 'second': None, 'depth1': None}


0: 640x640 (no detections), 239.0ms
Speed: 11.9ms preprocess, 239.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)



{'x_sort': None, 'currentClass': None, 'isEmpty': False, 'second': None, 'depth1': None}


...

{'x_sort': [array([ 28, 260])], 'currentClass': 5, 'isEmpty': False, 'second': 1705917146, 'depth1': [434]}


0: 640x640 (no detections), 416.0ms
Speed: 18.8ms preprocess, 416.0ms inference, 4.2ms postprocess per image at shape (1, 3, 640, 640)



{'x_sort': [array([ 28, 260])], 'currentClass': 5, 'isEmpty': False, 'second': 1705917146, 'depth1': [434]}


0: 640x640 (no detections), 234.4ms
Speed: 13.9ms preprocess, 234.4ms inference, 7.1ms postprocess per image at shape (1, 3, 640, 640)



{'x_sort': [array([ 28, 260])], 'currentClass': 5, 'isEmpty': False, 'second': 1705917146, 'depth1': [434]}
